INGESTÃO DE DADOS


In [13]:
import pandas as pd
import os

file_paths = {
    'admissions': 'rawcsvs/ADMISSIONS.csv',
    'microbiology_events': 'rawcsvs/MICROBIOLOGYEVENTS.csv',
    'prescriptions': 'rawcsvs/PRESCRIPTIONS.csv',
    'labevents': 'rawcsvs/LABEVENTS.csv',
    'diagnoses_icd': 'rawcsvs/DIAGNOSES_ICD.csv',
    'patients': 'rawcsvs/PATIENTS.csv',
    'transl-labitems': 'rawcsvs/D_LABITEMS.csv',
    'transl-diag': 'rawcsvs/D_ICD_DIAGNOSES.csv'
}

cols_to_use = {
    'admissions': ['subject_id','admittime','ethnicity', 'admission_type'],
    'microbiology_events': ['row_id', 'subject_id',  'chartdate', 'charttime', 'spec_itemid', 'spec_type_desc', 
                            'org_itemid', 'org_name', 'isolate_num', 'ab_itemid', 'ab_name', 'dilution_text', 
                            'dilution_comparison', 'dilution_value', 'interpretation'],
    'prescriptions': ['subject_id', "hadm_id", 'drug', 'enddate'],
    'labevents': ['subject_id', 'itemid', 'valuenum', 'valueuom', 'flag'],
    'diagnoses_icd': ['subject_id', 'icd9_code'], #gotta work on this one, adding all codes to one line in the array
    'transl-labitems': ['itemid', 'label'], # use this in conjunction with labevents to understando wtf if happening
    'patients': ['subject_id', 'gender'],
    'transl-diag': ['icd9_code', 'short_title', 'long_title']
}

files = {}
for file_path in file_paths:
    files[file_path] = pd.read_csv(file_paths[file_path], usecols=cols_to_use[file_path])
    
for file_path, file in files.items(): 
    if  not file_path.startswith('transl'):
        file.dropna(inplace=True, subset=['subject_id'])
        file.drop_duplicates(inplace=True)  
        file_name = os.path.basename(file_path)
        file.to_csv(f'cleanedcsv/{file_name}.csv', index=False)

MANEIRA PARA JUNTAR TODOS OS DADOS

In [2]:
'''
Isso esta feito para essas duas tabelas
o modo nao mudou, precisa mudar as tabelas importadas, as colunas pivoteadas, e o valor de value, dependendo do que 
precisa ser pivoteado
Agradeco Pedro por me dar a idea de pivotear as tabelas
'''
import pandas as pd
mergedata = pd.read_csv('mergedcsv/admin-diag-lab-micro.csv')
diagnoses = pd.read_csv('cleanedcsv/prescriptions.csv')


#inverter a tabela de diagnósticos para ter uma linha por paciente

diaginv = diagnoses.assign(value=1).pivot_table(
    index='subject_id', 
    columns='drug', 
    values='value', 
    fill_value=0
).reset_index()    # Reset the index to make `subject_id` a column

diaginv.fillna(0, inplace=True)

# Now proceed with the merge
mergedata = mergedata.merge(diaginv, on='subject_id', how='left')
mergedata.to_csv('mergedcsv/admin-diag-lab-micro-pat-prescrip.csv', index=False)




EXTRACAO DE DADOS IMPORTANTES A BUSCA


In [3]:
import Levenshtein
#supunhetemos que tenhamos uma busca
search = ab_name
#precisamos listar todas as doencas em que o medicamento foi usado
#para facilitar a busca, precisamos calcular o medicamento mais provavel que foi escrito
all_drugs = files['microbiology_events']['ab_name'].dropna().unique()

closest_drug = None	
for drug in all_drugs:
    dist = Levenshtein.distance(search, drug)
    if closest_drug is None or dist < closest_drug[1]:
        closest_drug = (drug, dist)

#agora que temos o medicamento mais provavel, podemos listar as resistencias



Coletando os dados de resistencia


In [ ]:
import plotly.express as px

#need to sort he diseases
diseases = files['microbiology_events'][files['microbiology_events']['ab_name'] == closest_drug[0]]['org_name'].unique()

dis_res_count = {}
for disease in diseases:
    disease_res = files['microbiology_events'][
        (files['microbiology_events']['ab_name'] == closest_drug[0]) & 
        (files['microbiology_events']['org_name'] == disease)
    ]['interpretation']
    
    # Initialize counts for 'R', 'I', 'S'
    counts = {'R': 0, 'I': 0, 'S': 0}
    
    counts.update(disease_res.value_counts().to_dict())
    
    dis_res_count[disease] = counts

plot = px.bar(x=list(dis_res_count.keys()), y=[v['R'] for v in dis_res_count.values()])
plot.show()    





FODASSE O MODELO PREDITIVO


In [15]:
from flask import Flask, render_template, request, jsonify
import Levenshtein


app = Flask(__name__)

# Rota para a página inicial (busca)
@app.route('/')
def home():
    return render_template('busca.html')
droga = None
# Rota para lidar com a busca de antibiótico (POST)
@app.route('/busca', methods=['POST'])
def busca():
    global droga
    # Captura o nome do antibiótico enviado via POST
    ab_name = request.form.get('ab_name')

    if ab_name:
        droga  = search(ab_name)
        return jsonify({"message": f"Antibiótico buscado: {droga}"})
    else:
        return jsonify({"error": "Nome do antibiótico não informado"}), 400

if __name__ == '__main__':
    app.run()

def search(term):
    all_drugs = files['microbiology_events']['ab_name'].dropna().unique()

    closest_drug = None	
    for drug in all_drugs:
        dist = Levenshtein.distance(term, drug)
        if closest_drug is None or dist < closest_drug[1]:
            closest_drug = (drug, dist)
    return closest_drug

# Rota para a página secundaria (resposta)
@app.route('/resposta')
def home():
    global droga
    if droga is not None:
        return render_template('resposta.html', closest_drug= droga )
    


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Oct/2024 18:23:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:05] "GET /static/estilos.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Oct/2024 18:23:05] "GET /static/busca.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Oct/2024 18:23:11] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:12] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:12] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:13] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:13] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:13] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:13] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:14] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:14] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:14] "POST /busca HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2024 18:23:15] "POST /busca HTTP/1.1" 200 -
127.

AssertionError: View function mapping is overwriting an existing endpoint function: home

In [14]:
import pandas as pd
mergedata = pd.read_csv('mergedcsv/admin-diag-lab-micro.csv')
diagnoses = pd.read_csv('cleanedcsv/prescriptions.csv')


#inverter a tabela de diagnósticos para ter uma linha por paciente

diaginv = diagnoses.assign(value=1).pivot_table(
    index='subject_id', 
    columns='drug', 
    values='value', 
    fill_value=0
).reset_index()    # Reset the index to make `subject_id` a column

diaginv.fillna(0, inplace=True)

# Now proceed with the merge
mergedata = mergedata.merge(diaginv, on='subject_id', how='left')
mergedata.to_csv('mergedcsv/admin-diag-lab-micro-pat-prescrip.csv', index=False)




SUGESTAO


In [66]:
def drug_suggestion(org_name, resistance):
    if resistance == 'R':
        return "Sugestão de antibiótico alternativo"
    else:
        return "Sugestão de antibiótico padrão"

merged_data['sugestao'] = merged_data.apply(lambda x: drug_suggestion(x['org_name'], x['interpretation']), axis=1)


FERRAMENTA VALIDAÇÃO


In [ ]:
from sklearn.metrics import accuracy_score

# Testar a ferramenta em um conjunto de dados de validação
resultados_reais = merged_data1['antibiotico_real']
sugestoes = merged_data1['sugestao']

acuracia = accuracy_score(resultados_reais, sugestoes)
print(f"Acurácia da ferramenta: {acuracia}")
